In [35]:
import laspy
import glob, os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

In [42]:
tree_files = glob.glob('/nas2/YJ/DATA/tree_normalized//training/velodyne/*.las')

In [43]:
len(tree_files)

319

In [44]:
# This is the config in kitti_dataset.yaml
# https://github.com/open-mmlab/OpenPCDet/blob/master/tools/cfgs/dataset_configs/kitti_dataset.yaml
'''
 POINT_CLOUD_RANGE: [0, -40, -3, 70.4, 40, 1]
   - NAME: transform_points_to_voxels
      VOXEL_SIZE: [0.05, 0.05, 0.1]
      MAX_POINTS_PER_VOXEL: 5
      MAX_NUMBER_OF_VOXELS: {
        'train': 16000,
        'test': 40000
      }
'''

"\n POINT_CLOUD_RANGE: [0, -40, -3, 70.4, 40, 1]\n   - NAME: transform_points_to_voxels\n      VOXEL_SIZE: [0.05, 0.05, 0.1]\n      MAX_POINTS_PER_VOXEL: 5\n      MAX_NUMBER_OF_VOXELS: {\n        'train': 16000,\n        'test': 40000\n      }\n"

In [47]:
# if voxel size is 0.16m, psuedo image be 375*375(60/0.16)
x_voxel_size = 0.0375
y_voxel_size = 0.0375
z_voxel_size = 0.8
x_range_min = -30.0
x_range_max = 30.0
y_range_min = -30.0
y_range_max = 30.0
z_range_min = 0.0
z_range_max = 32.0
normalized_value = 0.0

x_size = int((x_range_max-x_range_min)/x_voxel_size)
y_size = int((y_range_max-y_range_min)/y_voxel_size)
z_size = int((z_range_max-z_range_min)/z_voxel_size)
voxel = np.zeros((x_size, y_size, z_size))
print ('{}m, {}m and {}m of lidar ==> {}px, {}px and {}px of 3D voxel ==> {} locations'.format(
    x_range_max-x_range_min,
    y_range_max-y_range_min,
    z_range_max-z_range_min,
    voxel.shape[0],
    voxel.shape[1],
    voxel.shape[2],
    voxel.shape[0]*voxel.shape[1]*voxel.shape[2]
    ))

60.0m, 60.0m and 32.0m of lidar ==> 1600px, 1600px and 40px of 3D voxel ==> 102400000 locations


In [48]:
idxes = np.random.randint(len(tree_files), size=10)
voxels=[]
non_zeros = []
for idx in idxes :
    print (100*'#')
    print ('file: ', tree_files[idx])
    lasfile = laspy.file.File(tree_files[idx], mode="r")
    points = np.vstack((lasfile.x-30, lasfile.y-30
                         , lasfile.z-normalized_value)).transpose()
    print ('num_of_points: {}'.format(points.shape[0]))
    print ('range_of_x: {} to {}'.format(min(points[:,0]), max(points[:,0])))
    print ('range_of_y: {} to {}'.format(min(points[:,1]), max(points[:,1])))
    print ('range_of_z: {} to {}'.format(min(points[:,2]), max(points[:,2])))
    voxel = np.zeros((x_size, y_size, z_size))
    for point in points :
        if x_range_min <= point[0] <= x_range_max \
        and y_range_min <= point[1] <= y_range_max \
        and z_range_min <= point[2] <= z_range_max:
            r = int((point[0]-x_range_min - 0.0001)//x_voxel_size)
            c = int((point[1]-y_range_min - 0.0001)//y_voxel_size)
            d = int((point[2]-z_range_min - 0.0001)//z_voxel_size)
            voxel[r,c,d] +=1
    voxels.append(voxel)
    non_x, non_y, non_z = np.where(voxel != 0)
    non_zeros.append(non_x.shape[0])
    print ('non-zero_voxels: {} voxel'.format(non_x.shape[0]))
    print ('non-zero_ratio: {:.2f} %'.format(100* (non_x.shape[0]/(voxel.shape[0] * voxel.shape[1] * voxel.shape[2]))))
print ('average number of non_zeros: {:.2f}'.format(sum(non_zeros)/10.0))

####################################################################################################
file:  /nas2/YJ/DATA/tree_normalized//training/velodyne/T413.las
num_of_points: 604746
range_of_x: -30.01 to 30.009999999999998
range_of_y: -30.01 to 30.009999999999998
range_of_z: -1.47 to 15.11
non-zero_voxels: 469982 voxel
non-zero_ratio: 0.46 %
####################################################################################################
file:  /nas2/YJ/DATA/tree_normalized//training/velodyne/T1027.las
num_of_points: 493428
range_of_x: -30.01 to 30.009999999999998
range_of_y: -30.01 to 30.009999999999998
range_of_z: -1.87 to 11.57
non-zero_voxels: 429672 voxel
non-zero_ratio: 0.42 %
####################################################################################################
file:  /nas2/YJ/DATA/tree_normalized//training/velodyne/T1026.las
num_of_points: 629296
range_of_x: -30.01 to 30.009999999999998
range_of_y: -30.01 to 30.009999999999998
range_of_z: -0.36 to 13.18
n

In [49]:
non_zeros = []
for idx in range(len(tree_files)) :
    lasfile = laspy.file.File(tree_files[idx], mode="r")
    points = np.vstack((lasfile.x-30, lasfile.y-30
                         , lasfile.z-normalized_value)).transpose()
    voxel = np.zeros((x_size, y_size, z_size))
    for point in points :
        if x_range_min < point[0] < x_range_max \
        and y_range_min < point[1] < y_range_max \
        and z_range_min < point[2] < z_range_max:
            r = int((point[0]-x_range_min)//x_voxel_size)
            c = int((point[1]-y_range_min)//y_voxel_size)
            d = int((point[2]-z_range_min)//z_voxel_size)
            voxel[r,c,d] +=1
    non_x, non_y, non_z = np.where(voxel != 0)
    non_zeros.append(non_x.shape[0])
print ('average number of non_zeros: {:.2f}'.format(sum(non_zeros)/len(tree_files)))
print ('averae ratio of non_zeros: {:.2f}'.format(100*(sum(non_zeros)/len(tree_files))/(voxel.shape[0] * voxel.shape[1] * voxel.shape[2])))

average number of non_zeros: 139256.98
averae ratio of non_zeros: 0.14
